# Loading packages

In [12]:
import seaborn as sns
import collections
import os
import string
import utils.code as utils

from nltk.stem.snowball import SnowballStemmer

In [2]:
os.listdir('../../nlphw/gmb-2.2.0')

['data', 'doc', 'LICENSE', 'NEWS', 'README', 'THANKS']

In [2]:
corpus_root = '../../nlphw/gmb-2.2.0/data'

In [5]:
%%time

ner_tags = collections.Counter()
 
for root, dirs, files in os.walk(corpus_root):
    for filename in files:
        if filename.endswith(".tags"):
            with open(os.path.join(root, filename), 'rb') as file_handle:
                file_content = file_handle.read().decode('utf-8').strip()
                annotated_sentences = file_content.split('\n\n')   # Split sentences
                for annotated_sentence in annotated_sentences:
                    annotated_tokens = [seq for seq in annotated_sentence.split('\n') if seq]  # Split words
 
                    standard_form_tokens = []
 
                    for idx, annotated_token in enumerate(annotated_tokens):
                        annotations = annotated_token.split('\t')   # Split annotation
                        word, tag, ner = annotations[0], annotations[1], annotations[3]
 
                        # Get only the primary category
                        if ner != 'O':
                            ner = ner.split('-')[0]
 
                        ner_tags[ner] += 1

print( ner_tags )
# Counter({u'O': 1146068, u'geo': 58388, u'org': 48094, u'per': 44254, u'tim': 34789, u'gpe': 20680, u'art': 867, u'eve': 709, u'nat': 300})
 
print( "Words=", sum(ner_tags.values()) )

Counter({'O': 1146068, 'geo': 58388, 'org': 48094, 'per': 44254, 'tim': 34789, 'gpe': 20680, 'art': 867, 'eve': 709, 'nat': 300})
Words= 1354149
Wall time: 10min 9s


In [3]:
reader = read_gmb(corpus_root)

NameError: name 'read_gmb' is not defined

In [13]:
import os
for i in utils.read_gmb(corpus_root):
    print( i )

NameError: name 'os' is not defined